This notebook downloads daily state COVID-19 data from [The COVID Tracking Project](https://covidtracking.com/) and analyzes it.

### Set Up

Configures [DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/), HttpClient

In [59]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Net.Http;

// DataFrame formatter for Jupyter
Formatter<DataFrame>.Register((df, writer) => 
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent)th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 5;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(headers),
        tbody(rows.Select(r => tr(r)))
    );
    
    writer.Write(t);
}, "text/html");


var client = new HttpClient();

### Get Daily Data by State

[Model Documentation](https://covidtracking.com/api#apistatesdaily---states-historical-data)

In [62]:
string state = "MN";

DataFrame data;

// download the CSV and load it into a DataFrame
using (var response = await client.GetAsync("https://covidtracking.com/api/states/daily.csv?state=" + state))
using (var stream = await response.Content.ReadAsStreamAsync())
{
    data = DataFrame.LoadCsv(stream);
}

// parse the dateChecked column
var columns = data.Columns;
var dateChecked = columns["dateChecked"];
var newDateChecked = new PrimitiveDataFrameColumn<DateTimeOffset>("dateChecked", dateChecked.Length);
for (var i = 0; i < dateChecked.Length; i++)
{
    newDateChecked[i] = DateTimeOffset.Parse((string)dateChecked[i]);
}
columns["dateChecked"] = newDateChecked;

// remove some deprecated/unused columns
columns.Remove("total");
columns.Remove("hash");
columns.Remove("posNeg");

// show first five rows of loaded data
display(data);

0,20200408,MN,1242,31052,<null>,145,293,63,119,<null>,<null>,675,2020-04-09 20:00:00Z,50,293,32294,27,11,22,1453,88,1541
1,20200408,MN,1154,29599,<null>,135,271,64,105,<null>,<null>,632,2020-04-08 20:00:00Z,39,271,30753,27,5,29,1408,85,1493
2,20200408,MN,1069,28191,<null>,120,242,64,100,<null>,<null>,549,2020-04-07 20:00:00Z,34,242,29260,27,4,19,1049,83,1132
3,20200406,MN,986,27142,<null>,115,223,57,90,<null>,<null>,470,2020-04-06 20:00:00Z,30,223,28128,27,1,21,1300,51,1351
4,20200404,MN,935,25842,<null>,106,202,48,77,<null>,<null>,451,2020-04-05 20:00:00Z,29,202,26777,27,5,22,1284,70,1354


### Render Chart

In [63]:
#r "nuget:XPlot.Plotly"
using XPlot.Plotly;

var cases = new Graph.Scatter
{
    name = "Cases",
    x = data.Columns["dateChecked"],
    y = data.Columns["positive"]
};

var deaths = new Graph.Scatter
{
    name = "Deaths",
    x = data.Columns["dateChecked"],
    y = data.Columns["death"]
};

var chart = Chart.Plot(new[] { cases, deaths });
chart.WithTitle("Cases vs Deaths: " + state);
display(chart);